In [36]:
import cv2
import numpy as np
import mediapipe as mp

In [37]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [38]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [39]:
cap = cv2.VideoCapture(0)

In [40]:
# Pushup counter variables
counter = 0
stage = None
view = None
form_status = "Unknown"
warning_message = ""

In [41]:
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            if not landmarks:
                raise ValueError("No landmarks detected")

            # Calculate average visibilities for left and right sides
            left_vis = (
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility +
                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility +
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            ) / 3

            right_vis = (
                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility +
                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility +
                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            ) / 3

            # Determine view
            if left_vis > 0.6 and right_vis > 0.6:
                view = "front"
            elif left_vis > 0.6:
                view = "side_left"
            elif right_vis > 0.6:
                view = "side_right"
            else:
                view = "unknown"
                warning_message = "Improper placement or low light"
                form_status = "Unknown"
                cv2.putText(image, warning_message, (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                raise ValueError("Unknown view")

            # Check distance
            distance_status = ""
            if view == "front":
                shoulder_dist = abs(
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x -
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x
                )
                if shoulder_dist < 0.15:
                    distance_status = "Too far from camera"
                elif shoulder_dist > 0.6:
                    distance_status = "Too close to camera"
            elif view in ["side_left", "side_right"]:
                side_prefix = "LEFT" if view == "side_left" else "RIGHT"
                sh_idx = mp_pose.PoseLandmark[f"{side_prefix}_SHOULDER"].value
                hp_idx = mp_pose.PoseLandmark[f"{side_prefix}_HIP"].value
                sh = landmarks[sh_idx]
                hp = landmarks[hp_idx]
                torso_dist = np.sqrt((sh.x - hp.x)**2 + (sh.y - hp.y)**2)
                if torso_dist < 0.2:
                    distance_status = "Too far from camera"
                elif torso_dist > 0.7:
                    distance_status = "Too close to camera"

            if distance_status:
                warning_message = distance_status
                cv2.putText(image, warning_message, (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

            # Check form (body straight)
            good_form = False
            if view == "front":
                # Check both sides
                left_hip_angle = calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                )
                right_hip_angle = calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                )
                if abs(left_hip_angle - 180) < 30 and abs(right_hip_angle - 180) < 30:
                    good_form = True
            else:
                # Side view, check visible side
                side_prefix = "LEFT" if view == "side_left" else "RIGHT"
                sh_idx = mp_pose.PoseLandmark[f"{side_prefix}_SHOULDER"].value
                hp_idx = mp_pose.PoseLandmark[f"{side_prefix}_HIP"].value
                an_idx = mp_pose.PoseLandmark[f"{side_prefix}_ANKLE"].value
                hip_angle = calculate_angle(
                    [landmarks[sh_idx].x, landmarks[sh_idx].y],
                    [landmarks[hp_idx].x, landmarks[hp_idx].y],
                    [landmarks[an_idx].x, landmarks[an_idx].y]
                )
                if abs(hip_angle - 180) < 30:
                    good_form = True

            form_status = "Good" if good_form else "Bad - Straighten body"

            # Pushup counter logic
            if view == "front":
                avg_shoulder_y = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y + landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y) / 2
                avg_elbow_y = (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y + landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y) / 2
                if avg_shoulder_y > avg_elbow_y + 0.02:
                    stage = "down"
                if avg_shoulder_y < avg_elbow_y - 0.02 and stage == "down":
                    stage = "up"
                    if good_form:
                        counter += 1
                    print(counter)
            else:
                # Side view
                side_prefix = "LEFT" if view == "side_left" else "RIGHT"
                sh_idx = mp_pose.PoseLandmark[f"{side_prefix}_SHOULDER"].value
                el_idx = mp_pose.PoseLandmark[f"{side_prefix}_ELBOW"].value
                wr_idx = mp_pose.PoseLandmark[f"{side_prefix}_WRIST"].value

                shoulder = [landmarks[sh_idx].x, landmarks[sh_idx].y]
                elbow = [landmarks[el_idx].x, landmarks[el_idx].y]
                wrist = [landmarks[wr_idx].x, landmarks[wr_idx].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                # Visualize angle
                cv2.putText(image, str(int(angle)),
                            tuple(np.multiply(elbow, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                if angle < 100:
                    stage = "down"
                if angle > 150 and stage == "down":
                    stage = "up"
                    if good_form:
                        counter += 1
                    print(counter)

        except Exception as e:
            # print(e)
            pass

        # Render pushup counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (300, 100), (245, 117, 16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Stage data
        cv2.putText(image, 'STAGE', (95, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage if stage else "N/A",
                    (90, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Form data
        cv2.putText(image, 'FORM', (185, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, form_status,
                    (170, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
1
2
3
4
5
6
7
8
9
